In [ ]:
!pip install baseball_scraper
!pip install baseball_id

In [1]:
from baseball_scraper import espn, baseball_reference
from datetime import datetime
import pandas as pd
from baseball_scraper import fangraphs
from baseball_id import Lookup

In [ ]:
es = espn.ProbableStartersScraper(datetime(2021,7,21), datetime(2021,7,24))
es.scrape()

In [ ]:
tss = baseball_reference.TeamSummaryScraper()
# tss
# try:
tss.scrape(2021)
# except Exception as e:
#   print('test', e)
# df.columns

In [9]:
player_id = Lookup.from_names(['Khris Davis']).iloc[0].fg_id
# fangraphs.Scraper.instances()
fg = fangraphs.Scraper("THE BAT (RoS)")
df = fg.scrape(player_id, scrape_as=fangraphs.ScrapeType.PITCHER)
df.columns

ElementClickInterceptedException: Message: element click intercepted: Element <a id="ProjectionBoard1_cmdCSV" href="javascript:__doPostBack('ProjectionBoard1$cmdCSV','')">...</a> is not clickable at point (1409, 918). Other element would receive the click: <center>...</center>
  (Session info: chrome=92.0.4515.107)
